In [1]:
import pandas as pd

# Text vectorization :
from sklearn.feature_extraction.text import TfidfVectorizer

# Le modèle que l'on va entraîner :
from sklearn.svm import LinearSVC

# manipulation des données :
from sklearn.model_selection import train_test_split

import pickle
from scipy.sparse import hstack

## Importation des données

In [6]:
df = pd.read_excel('DATA.xlsx')
df.head()

,Objet,Expéditeur,Destinataire,Projet
0,"Merci d avoir modifié TEAMBER, Logiciel gestio...",noreply-maps-issues@google.com,alexis.cavaroc@teamber.fr,AGENCE DE COMMUNICATION
1,"Vielen Dank, dass Sie Änderungen für TEAMBER, ...",noreply-maps-issues@google.com,alexis.cavaroc@teamber.fr,AGENCE DE COMMUNICATION
2,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,l.bancel@labsoft.fr,DEVELOPPEMENT
3,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,teamber@teamber.fr,DEVELOPPEMENT
4,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,teamber@labsoft.fr,DEVELOPPEMENT


## fonctions rédigées pour la vectorisation du texte :

In [7]:
def fit_vectorize(X, col):
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(X)
    pickle.dump(tfidf_vectorizer,       open("TfidfVectorizer" + col,'wb'))
    return X_tfidf

def vectorize(X,col):
    tfidf_vectorizer = pickle.load(open("TfidfVectorizer"+col,'rb'))
    X_tfidf = tfidf_vectorizer.transform(X)
    return X_tfidf

def fit_transform_textcolumns(X, textcolumns):
    col = textcolumns[0]
    out = fit_vectorize(X[col],col)
    for i in range(1,len(textcolumns)) :
        col = textcolumns[i]
        out_next = fit_vectorize(X[col],col)
        out = hstack((out, out_next))
    return out

def transform_textcolumns(X, textcolumns):
    col = textcolumns[0]
    out = vectorize(X[col],col)
    for i in range(1,len(textcolumns)) :
        col = textcolumns[i]
        out_next = vectorize(X[col],col)
        out = hstack((out, out_next))
    return out

## Séparation du dataset en deux parties train/test :

In [12]:
cols_train = ['Objet','Expéditeur','Destinataire']
X_train, X_test, y_train, y_test = train_test_split(df[cols_train], df['Projet'],
                                                    test_size=.2, shuffle = True)

## Vectorisation des colonnes textuelles :

In [13]:
textcolumns = ["Objet",'Expéditeur','Destinataire']
X_train_vect= fit_transform_textcolumns(X_train,textcolumns)
X_test_vect = transform_textcolumns(X_test,textcolumns)

## Entraînement du modèle :

In [14]:
model = LinearSVC()
model.fit(X_train_vect,y_train)
print("Modèle entrainé sur : ",X_train_vect.shape[0],
      "mails\nPouvant être classés dans ",y_test.max(),
      'projets différents\nAccuracy : ',model.score(X_test_vect,y_test))

Modèle entrainé sur :  135806 mails
Pouvant être classés dans  ZUO PARIS projets différents
Accuracy :  0.9152627238454288


## Sauvegarde du modèle :

In [17]:
pickle.dump(model,open("MailClassifier",'wb'))

In [16]:
X_train_vect

<135806x10647 sparse matrix of type '<class 'numpy.float64'>'
	with 1641679 stored elements in COOrdinate format>